**[Advanced SQL Home Page](https://www.kaggle.com/learn/advanced-sql)**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,8e10063d8160fdda6799d345e440ca3166a9a33d,842c6b006fcddcc1eb059c64c6129ed5edd0971d894394...,2019-12-16 17:00:00+00:00,2019-12-16 17:00:00+00:00,205,0.20,None,None,None,None,...,1.0,5.25,Cash,Chicago Taxicab,None,None,None,None,None,None
1,8dd1f1fb249a6e1885bb70be21cb6dcc085e430b,084b422a6d13f5d6b7be4a235cb6cc68363b39f24536de...,2019-12-16 20:45:00+00:00,2019-12-16 21:00:00+00:00,600,1.70,None,None,None,None,...,0.0,7.50,Cash,Star North Management LLC,None,None,None,None,None,None
2,e912de7c03955048af1009381c86828e5134a8f0,c34d6347f812df0e22ee1a907881bffff574dab3839a95...,2015-06-27 01:45:00+00:00,2015-06-27 02:00:00+00:00,383,1.80,None,None,None,None,...,1.0,8.05,Cash,Checker Taxi,None,None,None,None,None,None
3,91682ca482808bb2abb5cd77c7b9a01f54e6371b,502a000e4ee99c96dc9964f2994671b57b73fd6902e234...,2021-11-20 10:00:00+00:00,2021-11-20 10:15:00+00:00,883,12.76,None,None,None,None,...,0.0,31.75,Cash,Flash Cab,None,None,None,None,None,None
4,8d7182ca3e092814bbff90ea671c6c6b62f60a2e,ec4b4f2f183bf84a4a4bd5b8df2ce0f94b98c05c06cb1a...,2019-12-03 17:15:00+00:00,2019-12-03 17:30:00+00:00,808,1.90,None,None,None,None,...,0.0,11.00,Credit Card,Metro Jet Taxi A,None,None,None,None,None,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to December 31, 2017.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 15 days and the following 15 days, as long as the days fit within the two-year time frame.  For instance, when calculating the value in this column for January 5, 2016, the window will include the number of trips for the preceding 4 days, the current date, and the following 15 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) micro-course.

In [5]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          avg(num_trips)
                          OVER (
                               order by trip_date
                               ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2017-09-26,10309.838710
1,2016-01-11,22338.692308
2,2016-07-24,18157.709677
3,2016-02-20,25006.000000
4,2017-07-10,15125.451613


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use the **AVG()** function. Write an **OVER** clause with that orders the rows with the `trip_date` column and uses a window that includes the 15 preceding rows, the current row, and the following 15 rows.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

```

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [6]:
# Amend the query below
trip_number_query = """
SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                    ""
# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp
0,41.0,2017-05-01 14:30:00+00:00,2017-05-01 14:30:00+00:00
1,32.0,2017-05-01 21:45:00+00:00,2017-05-01 21:45:00+00:00
2,28.0,2017-05-01 08:00:00+00:00,2017-05-01 08:00:00+00:00
3,32.0,2017-05-01 16:30:00+00:00,2017-05-01 16:45:00+00:00
4,32.0,2017-05-01 18:15:00+00:00,2017-05-01 18:30:00+00:00


<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> There are many different numbering functions that enumerate the rows in the input. For this exercise, please use the **RANK()** function.

In [14]:
# Lines below will give you a hint or solution code
#q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

```

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [12]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG (trip_end_timestamp, 1)
                            OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break, 
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """


# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,8562792428f3ba13f4f6810244df0278597616cd8b5d84...,2017-05-01 00:30:00+00:00,2017-05-01 01:00:00+00:00,NaN
1,883e86111c7bebb7bdd41cbcac8ebbcc1172fe7464b90f...,2017-05-01 00:00:00+00:00,2017-05-01 00:30:00+00:00,NaN
2,883e86111c7bebb7bdd41cbcac8ebbcc1172fe7464b90f...,2017-05-01 00:30:00+00:00,2017-05-01 00:45:00+00:00,0.0
3,883e86111c7bebb7bdd41cbcac8ebbcc1172fe7464b90f...,2017-05-01 01:45:00+00:00,2017-05-01 01:45:00+00:00,60.0
4,883e86111c7bebb7bdd41cbcac8ebbcc1172fe7464b90f...,2017-05-01 02:00:00+00:00,2017-05-01 02:15:00+00:00,15.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---
**[Advanced SQL Home Page](https://www.kaggle.com/learn/advanced-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*